In [65]:
import os
import runpy
import time
from os.path import join as pjoin
import warnings
import sys
import csv
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
from scipy import signal
from scipy.stats import zscore
from nilearn.image import load_img
from nilearn.masking import apply_mask
import matplotlib.pyplot as plt
import re
%matplotlib inline
%autosave 5

Autosaving every 5 seconds


In [3]:
!pwd

/Users/akalyani/Desktop/sense_map_juli


In [19]:
 ds_dir = '/Users/akalyani/Desktop/sense_map_juli/subjects'

In [4]:
def grab_subject_ids(ds_dir='/Users/akalyani/Desktop/sense_map_juli/subjects',
                     testsubs=False):
    """
    Get list of all subject IDs.
    """
    import os
    import glob
    sub_ids = [os.path.basename(subdir) for subdir in glob.glob(ds_dir + '/*')]
    if testsubs:
        sub_ids = sub_ids[:testsubs]
    return sub_ids
def path_grabber(roi_glm_workdir='/Users/akalyani/Desktop/sense_map_juli/subjects',
                prepped_dsdir='/Users/akalyani/Desktop/sense_map_juli/subjects',
                testsubs=False,
                excludesubs=()):
    """
    # grab file names for
    # filtered bold data and roi masks from roi_glm output
    """
    sub_ids = grab_subject_ids(testsubs=testsubs, ds_dir=prepped_dsdir)
    run1_data = []
    for sub_id in sub_ids:
        sub_dir = pjoin(roi_glm_workdir, sub_id)
        # file names for filtered bold files
        fname = pjoin(sub_dir, 'blocked_design', 'analyses')
        run1_data.append(fname)

    return run1_data

In [16]:
sub_ids = grab_subject_ids()

In [139]:
sub_ids

['bmz426', 'clz082', 'aae961', 'ben157']

In [5]:
F_paths =path_grabber()

In [6]:
# convert .img to .nii to be run only once
#for path in paths:
#    !fslchfiletype NIFTI $path/spmF_0001

In [7]:
"""importing labels"""


with open('/Volumes/Avinash PhD/cSRM/label_list.txt') as inf:
    reader = csv.reader(inf, delimiter="\t")
    labels = list(zip(*reader))[0]

In [15]:
labels

('lh.BA1.label',
 'lh.BA2.label',
 'lh.BA3a.label',
 'lh.BA3b.label',
 'lh.BA4a.label',
 'lh.BA4p.label',
 'lh.BA6.label',
 'rh.BA1.label',
 'rh.BA2.label',
 'rh.BA3a.label',
 'rh.BA3b.label',
 'rh.BA4a.label',
 'rh.BA4p.label',
 'rh.BA6.label')

In [8]:
#load the nifti files 1) labels 2)spmf_files
def masks_datagrabber(roi_glm_workdir='/Users/akalyani/Desktop/sense_map_juli/subjects',
                prepped_dsdir='/Volumes/Avinash PhD/cSRM/sense_data',
                testsubs=False,
                sub_ids=True):
    """
    # grab file names for
    # filtered bold data and roi masks from roi_glm output
    """
    sub_ids = grab_subject_ids(ds_dir ='/Users/akalyani/Desktop/sense_map_juli/subjects')
    #run1_masks, run2_masks= [], []
    run1_masks = list(np.zeros(len(sub_ids)))
    for indx,sub_id in enumerate(sub_ids):
        
        mask_dir = pjoin(prepped_dsdir,sub_id,'label_vol')
        masks=[]
        for lab in labels:
            mask_fname = pjoin(mask_dir, '%s.nii' %lab)
            masks.append(mask_fname)
            
        run1_masks[indx]= masks
        
    return run1_masks

In [9]:
label_masks=masks_datagrabber()

In [18]:
label_masks[0][1]

'/Volumes/Avinash PhD/cSRM/sense_data/bmz426/label_vol/lh.BA2.label.nii'

In [10]:
F_data=[pjoin(path,'spmF_0001.nii.gz') for path in F_paths]

In [37]:
#apply mask fslmultiply -> subject wise then check the output 
#if doesn't work then use apply mask, then convert back to nifti space 

In [13]:
F_paths

['/Users/akalyani/Desktop/sense_map_juli/subjects/bmz426/blocked_design/analyses',
 '/Users/akalyani/Desktop/sense_map_juli/subjects/clz082/blocked_design/analyses',
 '/Users/akalyani/Desktop/sense_map_juli/subjects/aae961/blocked_design/analyses',
 '/Users/akalyani/Desktop/sense_map_juli/subjects/ben157/blocked_design/analyses']

In [11]:
F_data

['/Users/akalyani/Desktop/sense_map_juli/subjects/bmz426/blocked_design/analyses/spmF_0001.nii.gz',
 '/Users/akalyani/Desktop/sense_map_juli/subjects/clz082/blocked_design/analyses/spmF_0001.nii.gz',
 '/Users/akalyani/Desktop/sense_map_juli/subjects/aae961/blocked_design/analyses/spmF_0001.nii.gz',
 '/Users/akalyani/Desktop/sense_map_juli/subjects/ben157/blocked_design/analyses/spmF_0001.nii.gz']

In [39]:
#THIS CELL TO BE RUN ONLY ONCE
# fslmaths input.nii.gz -mas mask.nii.gz out_label.nii.gz
for sub_idx,sub_id in enumerate(sub_ids):
    inp = F_data[sub_idx]
    out_path = pjoin(ds_dir,sub_id,'blocked_design/masked_labels')
    isExist = os.path.exists(out_path)
    if not isExist:
        os.makedirs(out_path)
        print("the masked_label directory is created")
    for m in range(14):
        mas = (label_masks[sub_idx][m])
        mask = "'%s'"%(mas)
        out_name = labels[m]
        out = pjoin(out_path,out_name)
        !fslmaths $inp -mas $mask $out
print('all DONE! MASKS EXTRACTED')       
        

In [167]:
#NOW thresholding and creating the binary mask 
# 1) number of Voxels 
# fslstats lh.BA2.label.nii.gz -V
all_sub_vox =[]
for sub_idx,sub_id in enumerate(sub_ids):
    inp = F_data[sub_idx]
    out_path = pjoin(ds_dir,sub_id,'blocked_design/masked_labels') 
    voxels = []
    for m in range(14):
        out_name = labels[m]
        out = pjoin(out_path,out_name+'.nii.gz')
        voxel = !fslstats $out -V
        #voxel is in list format so we form string first then split then take the first set of values
        a=''.join(voxel)
        s = []
        for t in a.split():
            try :
                s.append(float(t))
            except ValueError:
                pass
        
        # making a mask for the first significant 300 voxels
        prop = 500/s[0]
        per = 100-100*prop
        thresh = !fslstats $out -a -P $per
        
         #voxel is in list format so we form string first then split then take the first set of values
        th=''.join(thresh)
        threshold = []
        for t in th.split():
            try :
                threshold.append(float(t))
            except ValueError:
                pass
        TH = threshold[0]  
        bin_name = pjoin(out_path,out_name+'_bined.nii.gz')
        !fslmaths $out -thr $TH -bin $bin_name
        voxels.append(s[0])
       
        
    all_sub_vox.append(voxels)


In [169]:
n = np.min(all_sub_vox)

In [174]:
all_sub_vox[3]

[2538.0,
 3012.0,
 507.0,
 1806.0,
 6568.0,
 2402.0,
 8721.0,
 2696.0,
 3312.0,
 754.0,
 2041.0,
 6711.0,
 2851.0,
 10017.0]

In [ ]:
#checking for the whole brain fstat- as we want same number of voxels in our analysis, maybe 1000-2000 voxels
